In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import datetime
import joblib
from tensorflow.keras import utils
from keras.layers import Dense, Activation, Flatten, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [2]:
data_2017 = pd.read_csv("données/valeursfoncieres-2017-s2.csv", sep="|")
data_2018 = pd.read_csv("données/valeursfoncieres-2018.csv", sep="|")
data_2019 = pd.read_csv("données/valeursfoncieres-2019.csv", sep="|")
data_2020 = pd.read_csv("données/valeursfoncieres-2020.csv", sep="|")
data_2021 = pd.read_csv("données/valeursfoncieres-2021.csv", sep="|")
data_2022 = pd.read_csv("données/valeursfoncieres-2022-s1.csv", sep="|")

dataframes = []

C:\Users\xavid\AppData\Local\Temp\ipykernel_15980\490612154.py:1: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2017 = pd.read_csv("données/valeursfoncieres-2017-s2.csv", sep="|")
C:\Users\xavid\AppData\Local\Temp\ipykernel_15980\490612154.py:2: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2018 = pd.read_csv("données/valeursfoncieres-2018.csv", sep="|")
C:\Users\xavid\AppData\Local\Temp\ipykernel_15980\490612154.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2019 = pd.read_csv("données/valeursfoncieres-2019.csv", sep="|")
C:\Users\xavid\AppData\Local\Temp\ipykernel_15980\490612154.py:4: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2020 = 

In [3]:
dataframes.append(data_2017)
dataframes.append(data_2018)
dataframes.append(data_2019)
dataframes.append(data_2020)
dataframes.append(data_2021)
dataframes.append(data_2022)

In [4]:
data = pd.concat(dataframes, ignore_index=True)

In [5]:
data

,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/07/2017,Vente,...,NaN,0,1.0,Maison,NaN,73.0,4.0,S,NaN,391.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/07/2017,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/07/2017,Vente,...,NaN,1,2.0,Appartement,NaN,32.0,1.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/07/2017,Vente terrain à bâtir,...,NaN,0,NaN,NaN,NaN,NaN,NaN,AB,NaN,605.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/07/2017,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,AG,NaN,259.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18036807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
18036808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
18036809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/06/2022,Vente,...,NaN,1,2.0,Appartement,NaN,15.0,1.0,NaN,NaN,NaN
18036810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10/02/2022,Adjudication,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


In [6]:
data_c = data[['Nature mutation', 'Valeur fonciere', 'No voie', 'Code postal',
                         'Code departement', 'Code type local', 'Code voie', 'Surface reelle bati', 'Surface terrain']]

In [7]:
data_c.head()

,Nature mutation,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code voie,Surface reelle bati,Surface terrain
0,Vente,"166100,00",843.0,1000.0,1,1.0,0100,73.0,391.0
1,Vente,"59000,00",2.0,1000.0,1,3.0,1720,0.0,NaN
2,Vente,"59000,00",2.0,1000.0,1,2.0,1720,32.0,NaN
3,Vente terrain à bâtir,"55000,00",NaN,1160.0,1,NaN,B033,NaN,605.0
4,Vente,"130000,00",NaN,1750.0,1,NaN,B148,NaN,259.0


In [8]:
data_c["Code departement"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 22, 23, 24, 25, 26, 27, 28, 29, '29', '2A', '2B', '30', 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 971, 972, 973, 974, 75],
      dtype=object)

In [10]:
data_c.dropna(axis=0, inplace=True)
data_c= data_c.fillna(0)
data_c = data_c[data_c['Nature mutation'] == 'Vente']
data_c = data_c.drop(columns='Nature mutation')

In [11]:
data_c['Code postal'] = data_c['Code postal'].astype(int)
data_c['No voie'] = data_c['No voie'].astype(int)
data_c['Code postal'] = data_c['Code postal'].astype(str)
data_c['No voie'] = data_c['No voie'].astype(str)
data_c['Surface reelle bati'] = data_c['Surface reelle bati'].astype(str)
data_c["ID_Location"] = data_c['Code voie'].astype(str) +"-"+ data_c["Code postal"]+"-"+ data_c["No voie"]+"-"+ data_c["Surface reelle bati"]
data_c['Code postal'] = data_c['Code postal'].astype(float)
data_c['No voie'] = data_c['No voie'].astype(float)
data_c['Code departement'] = data_c['Code departement'].astype(str)
data_c['Code departement'] = data_c['Code departement'].str.replace('2A', '222')
data_c['Code departement'] = data_c['Code departement'].str.replace('2B', '223')
data_c = data_c.drop(columns='Code voie')

In [12]:
data_c["Code departement"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '222', '223', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '58', '59', '60', '61', '62', '63', '64', '65', '66', '69',
       '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '81',
       '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92',
       '93', '94', '95', '971', '972', '973', '974', '75'], dtype=object)

In [13]:
data_c['Valeur fonciere'] = data_c['Valeur fonciere'].str.replace(',', '.')
data_c['Valeur fonciere'] = data_c['Valeur fonciere'].astype(float)
data_c['Valeur fonciere'] = data_c['Valeur fonciere'].astype(int)
data_c = data_c[data_c['Valeur fonciere'] < 1000000]
data_c['Code type local'] = data_c['Code type local'].astype(int)
data_c['No voie'] = data_c['No voie'].astype(int)
data_c['Code postal'] = data_c['Code postal'].astype(int)
data_c['Code type local'] = data_c['Code type local'].astype(int)
#data_2021_c['Surface terrain'] = data_2021_c['Surface terrain'].astype(int)
data_c['Surface reelle bati'] = data_c['Surface reelle bati'].str.replace(',', '.')
data_c['Surface reelle bati'] = data_c['Surface reelle bati'].astype(float)
data_c['Surface reelle bati'] = data_c['Surface reelle bati'].astype(int)

In [14]:
data_c.set_index('ID_Location', inplace=True)

In [15]:
data_c

,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Surface reelle bati,Surface terrain
ID_Location,,,,,,,
0100-1000-843-73.0,166100,843,1000,1,1,73,391.0
B148-1750-5123-96.0,130000,5123,1750,1,1,96,471.0
3220-1000-3-72.0,145650,3,1000,1,2,72,165.0
B028-1560-5236-200.0,80000,5236,1560,1,1,200,731.0
B112-1380-5943-115.0,316500,5943,1380,1,1,115,956.0
...,...,...,...,...,...,...,...
9316-75015-14-53.0,320000,14,75015,75,2,53,152.0
9316-75015-14-52.0,320000,14,75015,75,2,52,152.0
9316-75015-14-53.0,320000,14,75015,75,2,53,152.0


In [16]:
data_c.sort_values(by=['ID_Location'], ascending=True, inplace=True)

In [17]:
data_c.dropna(axis=0, inplace=True)
data_c= data_c.fillna(0)
data_c = data_c.drop_duplicates()
data_c

,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Surface reelle bati,Surface terrain
ID_Location,,,,,,,
0001-1000-42-200.0,258200,42,1000,1,4,200,1451.0
0001-1000-67-370.0,285000,67,1000,1,4,370,1550.0
0001-10130-1-0.0,25000,1,10130,10,3,0,241.0
0001-10130-1-50.0,25000,1,10130,10,1,50,125.0
0001-10130-6-180.0,145000,6,10130,10,1,180,823.0
...,...,...,...,...,...,...,...
X721-73120-5459-0.0,663700,5459,73120,73,3,0,1430.0
X721-73120-5459-169.0,663700,5459,73120,73,1,169,1430.0
X912-73440-3-140.0,300000,3,73440,73,1,140,763.0


In [18]:
# Séparer les données en variables d'entrée (X) et variable cible (y)
X = data_c.drop("Valeur fonciere", axis=1)
y = data_c["Valeur fonciere"]

In [19]:
#data = data_2021_c.groupby(['ID_Location', 'Date mutation'])

In [20]:
#print(data.head(5))

In [21]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
#len(train_generator)

In [23]:
# Créer et entraîner le modèle GBM avec des hyperparamètres personnalisés
model = GradientBoostingRegressor(
    n_estimators=9000,  # Nombre d'estimateurs (arbres) dans l'ensemble
    learning_rate=0.05,  # Taux d'apprentissage
    max_depth=7,  # Profondeur maximale des arbres
    random_state=50  # Graine aléatoire pour la reproductibilité
)

In [24]:
# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=7, n_estimators=9000,
                          random_state=50)

In [25]:
y_pred = model.predict(X_test)

In [26]:
# Calculer l'erreur quadratique moyenne (RMSE) sur l'ensemble de test
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 105316.07829047609


In [27]:
print(y_pred)

[361864.09813159 151858.43037078  66123.8638982  ... 130260.8417007
  98749.97391926 223920.27678963]


In [28]:
# Enregistrer le modèle
joblib.dump(model, 'GBM_Model_full.pkl')

['GBM_Model_full.pkl']